In [57]:
#import the neccessary packages
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import gradio as gr
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import Document


In [58]:
#Suppress known warning related to LangChain version compatibility
import warnings
warnings.filterwarnings(
    "ignore",
    message="Convert_system_message_to_human will be deprecated!",
    module="langchain_google_genai.chat_models"
)

In [59]:
#Initialize the Gemini LLM with API key
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key="PASTE YOUR API KEY HERE", # (Note: Avoid hardcoding this in real apps)
    convert_system_message_to_human=True
)

text_splitter = CharacterTextSplitter()

In [60]:
#Summarization logic function
def summarize_input(file, direct_text, custom_prompt=""):
    docs = []
    # Case 1: File uploaded
    if file is not None:
        file_path = file.name
        if file_path.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
            docs = loader.load_and_split()
        elif file_path.endswith(".txt"):
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read()
            docs = text_splitter.split_documents([Document(page_content=text)])
    # Case 2: Direct text input instead of file
    elif direct_text.strip() != "":
        docs = text_splitter.split_documents([Document(page_content=direct_text)])

    # If no valid input
    if not docs:
        return "No input provided.", ""

    # Default prompt for summarization
    default_prompt_template = """
    You are an expert summarizer.
    Summarize the following document clearly and concisely using bullet points or short paragraphs. Avoid vague language.
    {text}
    SUMMARY:
    """
     # Wrap default prompt in LangChain's PromptTemplate
    DEFAULT_PROMPT = PromptTemplate(template=default_prompt_template, input_variables=["text"])
     # Build summarization chain using MapReduce approach
    chain = load_summarize_chain(llm, chain_type="map_reduce", map_prompt=DEFAULT_PROMPT, combine_prompt=DEFAULT_PROMPT)
    # Run chain with document chunks and extract output
    summary = chain.invoke({"input_documents": docs})["output_text"]

    # Optional: If user gives custom prompt
    custom_summary = ""
    if custom_prompt.strip() != "":
        prompt_template = custom_prompt + """
        {text}
        SUMMARY:"""
        PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
        chain = load_summarize_chain(llm, chain_type="map_reduce", map_prompt=PROMPT, combine_prompt=PROMPT)
        custom_summary = chain.invoke({"input_documents": docs})["output_text"]

    return summary, custom_summary

In [61]:
# Define the Gradio interface
def main():
    file_input = gr.File(file_types=[".pdf", ".txt"], label="Upload PDF or TXT File")
    direct_text_input = gr.Textbox(label="Or enter text directly", lines=10, placeholder="Paste your text here...")
    input_custom_prompt = gr.Textbox(label="Enter your custom prompt (optional)")
    output_summary = gr.Textbox(label="Summary", lines=10)
    output_custom_summary = gr.Textbox(label="Custom Summary", lines=10)

    # Create the Gradio interface
    iface = gr.Interface(
        fn=summarize_input,
        inputs=[file_input, direct_text_input, input_custom_prompt],
        outputs=[output_summary, output_custom_summary],
        title="Text Summarizer",
        description="Upload a PDF or TXT file, or paste text directly to get a summary.",
    )

    iface.launch() # Run the web app

# entry point for script
if __name__ == "__main__":
    main()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.
